In [1]:
import tensorflow as tf
import numpy as np
import os
import random

In [2]:
def error_measure(predictions, labels):
    """ calculate sum squared error of predictions """
    mistakes = tf.not_equal(tf.argmax(labels, 1), tf.argmax(predictions, 1))
    return tf.reduce_mean(tf.cast(mistakes, tf.float32))
    
def dense_to_one_hot(labels_dense, num_classes=10):
    """ Convert class labels from scalars to one-hot vectors. """
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

def get_xy(start, end, vid, labels, feat):
    inp, output = [], []
    for i in range(start,end+1):
        tmp = []
        # check if number of frame in the current video are greater than 16
        if len(feat[vid[i]]) > 16:
            current_frame = random.randrange(0,len(feat[vid[i]])-16)
            c_start = 0
            c_end = current_frame+16
            
            for frame in sorted(feat[vid[i]]):
                if (c_start > current_frame) and (c_end > current_frame):
                    current_frame += 1
                    tmp.append(feat[vid[i]][frame].flatten())
                c_start += 1
        else:
            for frame in sorted(feat[vid[i]]):
                tmp.append(feat[vid[i]][frame].flatten())
            last_frame_id = sorted(feat[vid[i]])[-1]
            while len(tmp) <= 15:
                tmp.append(feat[vid[i]][last_frame_id].flatten())
        
        inp.append(tmp)
        output.append(labels[vid[i]])
    output = dense_to_one_hot(np.array(output), n_classes)

    return inp, output

In [3]:
n_epoch = 100
batch_size = 128
display_step = 10

n_input = 4096
n_steps = 16
n_hidden = 128
n_classes = 7

logs_path = 'logs/'
subset = 'normal'
root = 'backup/'

In [4]:
feat_train = np.load(root+'feat_train.npy', encoding='latin1').item(0)
feat_val = np.load(root+'feat_val.npy', encoding='latin1').item(0)

vid_train = np.load(root+'vid_train.npy')
vid_val = np.load(root+'vid_val.npy')

labels_train = np.load(root+'labels_train.npy').item(0)
labels_val = np.load(root+'labels_val.npy').item(0)

In [5]:
def lstm_model(subset)
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, n_steps, n_input])
        y = tf.placeholder(tf.float32, [None, n_classes])

    with tf.name_scope('lstm_cell'):
        cell = tf.contrib.rnn.LSTMCell(n_hidden)
        val, _ = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
        val = tf.transpose(val, [1, 0, 2])
        last = tf.gather(val, int(val.get_shape()[0]) - 1)

    with tf.name_scope('weight'):
    #     weight = tf.get_variable(name='weight',shape= [n_hidden, int(y.get_shape()[1])], initializer=tf.contrib.layers.xavier_initializer())
        weight = tf.get_variable(name='weight',shape= [n_hidden, int(y.get_shape()[1])], initializer=tf.truncated_normal_initializer(stddev=0.1))
    with tf.name_scope('bias'):
    #     bias = tf.get_variable(name='bias',shape=[y.get_shape()[1]], initializer=tf.contrib.layers.xavier_initializer())
        bias = tf.get_variable(name='bias',shape=[y.get_shape()[1]], initializer=tf.truncated_normal_initializer(stddev=0.1))

    with tf.name_scope('softmax'):
        pred = tf.nn.softmax(tf.matmul(last, weight) + bias)

    with tf.name_scope('cross_entropy'):
    #     cross_entropy = -tf.reduce_sum(y * tf.log(tf.clip_by_value(pred,1e-10,1.0)))
    #     cross_entropy = tf.reduce_mean(-tf.reduce_sum(pred * tf.log(y), reduction_indices=[1]))
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer()
        minimize = optimizer.minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    return 


/usr/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
init = tf.global_variables_initializer()
# saver = tf.train.Saver()

# create a summary for cross_entropy and accuracy
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.summary.merge_all()

In [7]:
with tf.Session() as sess:
    sess.run(init)
    no_of_batches = int((len(feat_train)) / batch_size)
    train_writer = tf.summary.FileWriter('%strain_%s/'%(logs_path, subset), sess.graph)
    val_writer = tf.summary.FileWriter('%sval_%s/'%(logs_path, subset), sess.graph)

    for i in range(n_epoch):
        ptr = 0
        np.random.shuffle(vid_train)
        for j in range(no_of_batches):
            inp, out = get_xy(ptr,ptr+batch_size, vid_train, labels_train, feat_train)
            sess.run([minimize],{x: inp, y: out})
            
        r = random.randrange(0, len(feat_train)-300)
        train_input, train_output = get_xy(0,len(feat_train)-1, vid_train, labels_train, feat_train)
        val_input, val_output = get_xy(0,len(feat_val)-1, vid_val, labels_val, feat_val)
                
        loss_trian,  acc_train, summary_train, props_train = sess.run([cross_entropy, accuracy, summary_op, pred],{x: train_input, y: train_output})
        train_writer.add_summary(summary_train,i)
        
        _, acc_val, summary_val, props_val = sess.run([cross_entropy, accuracy, summary_op, pred],{x: val_input, y: val_output})
        val_writer.add_summary(summary_val,i)
        # Save the variables to disk.
#         if i %  25 == 0:
#             save_path = saver.save(sess, "backup/model_%d.ckpt"%i)
#             print("Model saved in file: %s" % save_path)       
        print("Epoch ",str(i), ', acc_train: %.7f' % (acc_train*100), 'acc_val: %.7f' % (acc_val*100), ', loss_trian: %.7f' % loss_trian)

Epoch  0 , acc_train: 30.8201045 acc_val: 19.6765497 , loss_trian: 1.8831360
Epoch  1 , acc_train: 36.7724866 acc_val: 25.6064683 , loss_trian: 1.8426372
Epoch  2 , acc_train: 39.5502657 acc_val: 26.9541770 , loss_trian: 1.8149766
Epoch  3 , acc_train: 43.5185194 acc_val: 30.1886797 , loss_trian: 1.7835377
Epoch  4 , acc_train: 39.9470896 acc_val: 28.5714298 , loss_trian: 1.7896413
Epoch  5 , acc_train: 42.5925940 acc_val: 29.3800533 , loss_trian: 1.7622049
Epoch  6 , acc_train: 46.5608478 acc_val: 33.6927235 , loss_trian: 1.7438910
Epoch  7 , acc_train: 46.5608478 acc_val: 30.1886797 , loss_trian: 1.7449830
Epoch  8 , acc_train: 50.9259284 acc_val: 33.9622647 , loss_trian: 1.7243435
Epoch  9 , acc_train: 49.7354507 acc_val: 32.8840971 , loss_trian: 1.7159246
Epoch  10 , acc_train: 50.9259284 acc_val: 29.9191386 , loss_trian: 1.6970786
Epoch  11 , acc_train: 51.4550269 acc_val: 30.4582208 , loss_trian: 1.6965632
Epoch  12 , acc_train: 53.0423284 acc_val: 34.2318058 , loss_trian: 1.6810